# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper Parameters

In [3]:
hyper_parameters = {
    "SEED":1990,
    "notebookName":"ComplexModel2",
    "nepochs":10000,
    "batch_size":256,
    "learning_rate":0.001,
    "window_size":128,
    "max_patience_count":1000
}

In [4]:
torch.manual_seed(hyper_parameters["SEED"])

In [5]:
f"./models/{hyper_parameters['notebookName']}"

'./models/ComplexModel2'

In [6]:

PATH = Path(f"./models/{hyper_parameters['notebookName']}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files = 2

# Set Path

In [7]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Help Functions

# Load Data

In [8]:
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))

In [9]:
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [10]:
for col in df_train_default.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
TimeNanos
LeapSecond
FullBiasNanos
BiasNanos
BiasUncertaintyNanos
DriftNanosPerSecond
DriftUncertaintyNanosPerSecond
HardwareClockDiscontinuityCount
Svid
TimeOffsetNanos
State
ReceivedSvTimeNanos
ReceivedSvTimeUncertaintyNanos
Cn0DbHz
PseudorangeRateMetersPerSecond
PseudorangeRateUncertaintyMetersPerSecond
AccumulatedDeltaRangeState
AccumulatedDeltaRangeMeters
AccumulatedDeltaRangeUncertaintyMeters
CarrierFrequencyHz
MultipathIndicator
ConstellationType
AgcDb
BasebandCn0DbHz
FullInterSignalBiasNanos
FullInterSignalBiasUncertaintyNanos
SatelliteInterSignalBiasNanos
SatelliteInterSignalBiasUnc

# Dataloader

In [11]:
df_train_default['phone'].value_counts()

2021-04-22-US-SJC-1_Pixel4             2890
2021-04-22-US-SJC-1_SamsungS20Ultra    2826
2020-09-04-US-SF-2_Mi8                 2500
2021-04-29-US-SJC-2_SamsungS20Ultra    2370
2020-09-04-US-SF-2_Pixel4              2349
                                       ... 
2021-01-05-US-SVL-2_Pixel4XL           1193
2020-06-05-US-MTV-1_Pixel4XLModded     1123
2021-04-26-US-SVL-1_Mi8                1036
2021-04-26-US-SVL-1_Pixel5             1034
2020-05-14-US-MTV-2_Pixel4XLModded      577
Name: phone, Length: 73, dtype: int64

In [12]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes))
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train)
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid)
    
    return df_train.reset_index().drop(columns = 'index'), df_valid.reset_index().drop(columns = 'index')
    
df_train, df_valid = CustomTrainValidSplit(df_train_default, valid_size = 0.1)
print(df_train.shape, df_valid.shape)
    

(119702, 148) (12855, 148)


In [13]:
df_train

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,GPS_L1,GPS_L5,GAL_E1,GAL_E5A,GLO_G1,BDS_B1I,BDS_B1C,BDS_B2A,QZS_J1,QZS_J5
0,2021-04-22-US-SJC-1,Pixel4,1303160575446,37.334582,-121.899408,-10.66,2021-04-22-US-SJC-1_Pixel4,325.45,1.80,0.0,...,0,0,0,0,1,0,0,0,0,0
1,2021-04-22-US-SJC-1,Pixel4,1303160576446,37.334603,-121.899416,-6.76,2021-04-22-US-SJC-1_Pixel4,326.45,1.80,0.0,...,0,0,1,0,0,0,0,0,0,0
2,2021-04-22-US-SJC-1,Pixel4,1303160577446,37.334579,-121.899418,-10.73,2021-04-22-US-SJC-1_Pixel4,327.45,1.80,0.0,...,0,0,0,0,0,1,0,0,0,0
3,2021-04-22-US-SJC-1,Pixel4,1303160578446,37.334595,-121.899419,-6.35,2021-04-22-US-SJC-1_Pixel4,328.45,1.80,0.0,...,0,1,0,0,0,0,0,0,0,0
4,2021-04-22-US-SJC-1,Pixel4,1303160579446,37.334590,-121.899406,-19.56,2021-04-22-US-SJC-1_Pixel4,329.45,1.80,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119697,2021-04-28-US-SJC-1,Pixel4,1303677188438,37.334252,-121.900218,26.54,2021-04-28-US-SJC-1_Pixel4,2361.44,3.10,0.0,...,0,0,1,0,0,0,0,0,0,0
119698,2021-04-28-US-SJC-1,Pixel4,1303677189438,37.334186,-121.900204,-9.88,2021-04-28-US-SJC-1_Pixel4,2362.44,3.10,0.0,...,0,1,0,0,0,0,0,0,0,0
119699,2021-04-28-US-SJC-1,Pixel4,1303677190438,37.334274,-121.900219,27.98,2021-04-28-US-SJC-1_Pixel4,2363.44,3.10,0.0,...,1,0,0,0,0,0,0,0,0,0
119700,2021-04-28-US-SJC-1,Pixel4,1303677191438,37.334273,-121.900206,17.66,2021-04-28-US-SJC-1_Pixel4,2364.44,3.10,0.0,...,0,0,0,1,0,0,0,0,0,0


In [14]:
def GetWindows(idx, window_size):
    index = np.array([])
    if idx < window_size:
        index = np.concatenate([np.zeros(window_size - idx-1), np.array(range(idx+1))])
        pass
    else:
        index = np.array(range(idx-window_size+1, idx+1))
    return index.astype(int)

def GetWindowsWithRatio(idx, max_idx, window_size, window_ratio = 1.):
    left_index = np.array([])
    right_index = np.array([])
    
    left_size = int(round(window_size * window_ratio))
    right_size = window_size - left_size
    if idx - left_size< 0:
        left_index = np.concatenate([np.zeros(left_size - idx-1), np.array(range(idx+1))])
    else:
        left_index = np.array(range(idx-left_size, idx+1))
    
    if idx + right_size> max_idx:
        right_index = np.concatenate([np.array(range(idx+1, max_idx+1)), (max_idx)*np.ones(right_size - idx - 1)])
    else:
        right_index = np.array(range(idx+1, idx + right_size))
    
    index = np.concatenate([left_index, right_index])
    
    if index.shape[0] < window_size:
        if idx > np.percentile(index, window_ratio * 100):
            if index[-1] == max_idx:
                addtional_index = index[-1]
            else:
                addtional_index = index[-1] + 1
            index = np.concatenate([index, np.array([addtional_index])])
        
    return index.astype(int)


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame, 
                 features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'], 
                 labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM'],
                 window_size = 100,
                 train = False,
                 augment = False,
                 crop_ratio = 0.0,
                device = 'cpu'):
        self.df = df
        self.features = features
        self.labels = labels
        self.len = df.shape[0]
        self.window_size = window_size
        self.train = train
        self.augment = augment
        self.crop_ratio = crop_ratio
        self.device = device
        
        self.data = self.df[features].astype(float).values
        if train == True:
            self.true = self.df[labels].astype(float).values
        else:
            self.true = []
        self.phone = self.df['phone'].values
        self.millisSinceGpsEpoch = self.df['millisSinceGpsEpoch'].values
        
        self.start_index_by_phone = dict()
        self.length_by_phone = dict()
        
        for phone in set(self.phone):
            start_index = np.where(self.phone == phone)[0][0]
            self.start_index_by_phone[phone] = start_index
            self.length_by_phone[phone] = (self.phone == phone).sum().astype('int64')
            
        
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        # 필요하다면 random crop 코드도 추가(랜덤한 위치에서 일정 비율의 데이터를 0으로 초기화)
        phone = self.phone[idx]
        start_index = self.start_index_by_phone[phone]
        
#         window_index = GetWindowsWithRatio(idx - start_index, self.length_by_phone[phone], self.window_size, 1) + start_index
        window_index = GetWindows(idx - start_index, self.window_size) + start_index
#         data = self.data[window_index, :].transpose(1, 0) # num_of_features X window_size
        data = self.data[window_index, :]                   # window_size X num_of_features
        
            
        indx = [self.phone[idx], self.millisSinceGpsEpoch[idx]]
        
        # data shape : num_of_features X window_size
        # true shape : num_of_labels X 1
        data = torch.Tensor(data)
        if self.augment:
            crop_size = int(self.crop_ratio * self.window_size * np.random.rand(1))
            crop_start_index = np.random.randint(0, self.window_size - crop_size)
            data[crop_start_index:crop_size] = 0
        
        if self.train is False:
            true = []
        else:
            true = self.true[idx]
            true = torch.Tensor(true.astype(float))
        
        return data, true, indx
    


In [15]:
features = [
    'latDeg', 
    'lngDeg', 
    'heightAboveWgs84EllipsoidM',
    'dlatDeg_Scaled',
    'dlngDeg_Scaled',
    'dheight_Scaled',
    'xSatPosM_Scaled',
    'ySatPosM_Scaled',
    'zSatPosM_Scaled',
    'xSatVelMps_Scaled',
    'ySatVelMps_Scaled',
    'zSatVelMps_Scaled',
    'UncalGyroXRadPerSec_Scaled',
    'UncalGyroYRadPerSec_Scaled',
    'UncalGyroZRadPerSec_Scaled',
    'DriftXRadPerSec_Scaled',
    'DriftYRadPerSec_Scaled',
    'DriftZRadPerSec_Scaled',
    'UncalAccelXMps2_Scaled',
    'UncalAccelYMps2_Scaled',
    'UncalAccelZMps2_Scaled',
    'BiasXMps2_Scaled',
    'BiasYMps2_Scaled',
    'BiasZMps2_Scaled',
    'UncalMagXMicroT_Scaled',
    'UncalMagYMicroT_Scaled',
    'UncalMagZMicroT_Scaled',
#   'BiasXMicroT_Scaled',
#   'BiasYMicroT_Scaled',
#   'BiasZMicroT_Scaled',
    'yawDeg_Scaled',
    'rollDeg_Scaled',
    'pitchDeg_Scaled',
    'GPS_L1', 
    'GPS_L5', 
    'GAL_E1', 
    'GAL_E5A', 
    'GLO_G1', 
    'BDS_B1I', 
    'BDS_B1C', 
    'BDS_B2A', 
    'QZS_J1', 
    'QZS_J5'
]
status_features = [
]

labels = [
    't_latDeg', 
    't_lngDeg', 
    't_heightAboveWgs84EllipsoidM'
         ]

print(df_train[features].shape)
print(df_train[features].describe())


(119702, 40)
              latDeg         lngDeg  heightAboveWgs84EllipsoidM  \
count  119702.000000  119702.000000               119702.000000   
mean       37.422746    -122.122703                   22.246878   
std         0.089786       0.165307                   60.534126   
min        37.321683    -122.429130                -6157.470000   
25%        37.352091    -122.263294                  -13.370000   
50%        37.391921    -122.088943                    4.640000   
75%        37.467238    -122.060264                   48.460000   
max        37.690836    -121.881855                 1185.730000   

       dlatDeg_Scaled  dlngDeg_Scaled  dheight_Scaled  xSatPosM_Scaled  \
count   119702.000000   119702.000000   119702.000000    119702.000000   
mean        -0.067276        0.076099       -0.026404         0.004041   
std          0.960498        0.960428        0.853825         0.981131   
min         -3.367212       -2.556995      -84.852009        -1.742306   
25%         -

In [16]:
train_data = CustomDataset(df_train, 
                           features = features, labels = labels, 
                           window_size = hyper_parameters['window_size'], 
                           train = True, 
                           augment = True,
                           crop_ratio = 0.1,
                           device = device)
valid_data = CustomDataset(df_valid, 
                           features = features, labels = labels, 
                           window_size = hyper_parameters['window_size'], 
                           train = True, 
                           augment = False,
                           crop_ratio = 0.0,
                           device = device)
test_data = CustomDataset(df_test, 
                        features = features, labels = labels, 
                        window_size = hyper_parameters['window_size'], 
                        train = False, 
                        augment = False,
                        crop_ratio = 0.0,
                        device = device)

In [17]:
train_loader = DataLoader(train_data, batch_size = hyper_parameters['batch_size'], shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = hyper_parameters['batch_size'], shuffle = False)
test_loader = DataLoader(test_data, batch_size = hyper_parameters['batch_size'], shuffle = False)

# Build Model

In [18]:
class GPSLossScore(nn.Module):
    def __init__(self):
        super().__init__()
        self.EARTH_RADIUS = 6_367_000
        self.loss = nn.SmoothL1Loss()
        pass
    
    def forward(self, predict:torch.Tensor, target:torch.Tensor):
        
#         loss = (0.5) * self.gps_loss(predict, target) + (0.5) * self.EARTH_RADIUS * self.mse(predict, target)
#         loss = self.gps_loss(predict, target)
        loss = self.loss(predict, target)
        return loss
    
    def torch_haversine(self,lat1, lon1, lat2, lon2):
    
        lat1=lat1 % 360
        lon1=lon1 % 360
        lat2=lat2 % 360
        lon2=lon2 % 360

        lat1, lat2, lon1, lon2 = map(torch.deg2rad, [lat1, lat2, lon1, lon2])

        dlat = (lat2 - lat1)
        dlon = (lon2 - lon1)

        a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2.0)**2)
        c = 2 * torch.arcsin(a ** 0.5)

        dist = self.EARTH_RADIUS * c

        return dist

    def gps_loss(self, predict:torch.Tensor, target:torch.Tensor):
        dist = self.torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

        loss = dist.mean()

        return loss

    def gps_score(self, predict:torch.Tensor, target:torch.Tensor):
        dist = self.torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

        score = (torch.quantile(dist, 0.5) + torch.quantile(dist, 0.95))/2

        return score
    

In [19]:

class ConvBlock(nn.Module):
    def __init__(self, input_features, features = 128):
        super().__init__()
        
        self.fuse = nn.Conv1d(input_features + features, features, kernel_size = 1)
        self.conv = nn.Conv1d(input_features, features, kernel_size=3, padding = 1)
        self.batch = nn.BatchNorm1d(features)
        self.pool = nn.AvgPool1d(kernel_size = 2)
        
    def forward(self, x):
        skip = x
        
        x = self.conv(x)
        x = F.tanh(x)
        x = self.batch(x)
        x = torch.cat([skip, x], axis = 1)
        x = self.fuse(x)
        x = self.pool(x)
        return x
    
class BaseModel(nn.Module):
    def __init__(self, input_size = (100, 3), output_size = 3):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.conv1 = ConvBlock(input_size[1], 64)
        self.conv2 = ConvBlock(64, 64)
        self.conv3 = ConvBlock(64, 64)
        
        self.tf = nn.Transformer(d_model = 64, 
                                 nhead = 32, 
                                 num_encoder_layers = 1, 
                                 num_decoder_layers = 1, 
                                 dim_feedforward = 64,
                                 dropout = 0.3, 
                                 activation= 'relu')
        
        self.batch_norm = nn.BatchNorm1d(64)
        self.layer_norm = nn.LayerNorm([16, 64])
        
        self.fc1 = nn.Linear(16*64, 128)
        self.layer_norm1 = nn.LayerNorm(128)
        self.fc2 = nn.Linear(128, 64)
        self.layer_norm2 = nn.LayerNorm(64)
        self.fc3 = nn.Linear(64, 32)
        self.layer_norm3 = nn.LayerNorm(32)
        self.fc4 = nn.Linear(32, output_size)
        self.layer_norm4 = nn.LayerNorm(3)
        
        self.drop06 = nn.Dropout(0.6)
        self.drop03 = nn.Dropout(0.3)
        self.drop01 = nn.Dropout(0.1)
        
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.kaiming_normal_(self.fc3.weight)
        nn.init.kaiming_normal_(self.fc4.weight)
        
        
    def forward(self, x):
        input_size = self.input_size 
        output_size = self.output_size
        
        x[:,:,:2] = torch.deg2rad(x[:,:,:2])
        
        x = x.transpose(2,1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.transpose(2,1)
        
        skip = x
        x = self.layer_norm(x)
        x = self.tf(x, x)
        x = self.batch_norm(x.transpose(2,1))
        x = x.transpose(2,1)
        x = x + skip
        
        x = x.reshape(-1, 16*64)
        
        x = self.drop06(x)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.layer_norm1(x)
        
        x = self.drop06(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.layer_norm2(x)
        
        x = self.drop03(x)
        x = self.fc3(x)
        x = F.tanh(x)
        x = self.layer_norm3(x)
        
        x = self.drop01(x)
        x = self.fc4(x)
        x = self.layer_norm4(x)
        
        x[:,:2] = torch.rad2deg(x[:,:2])
        
        return x

In [20]:
model = BaseModel((hyper_parameters['window_size'], len(features)), len(labels))
model.to(device)
# model.load_state_dict(torch.load("./models/Baseline3/model-4.pth"))

# loss_func = nn.SmoothL1Loss()
loss_func = GPSLossScore()
optimizer = optim.Adam(model.parameters(), lr = hyper_parameters['learning_rate'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.1,
                                                patience = 5,
                                                verbose = True)

# Fit and Validate

In [21]:
def train(epoch, progress_log, status_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    for data, targets, _ in progress_log:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)
        
        status_log.set_description_str(f"train status: loss - {loss_func(outputs, targets)} | {loss_func.gps_loss(outputs, targets)}, score - {loss_func.gps_score(outputs, targets)}")

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = loss_func.gps_score(predict, ground)
    
    scheduler.step(loss)
    return loss, meas

In [22]:
def valid(progress_log, status_log):
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets, _ in progress_log:
            
            data = data.to(device)
            targets = targets.to(device)
            
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
            
            predict.append(outputs)
            ground.append(targets)
            
        status_log.set_description_str(f"valid status: loss - {loss_func(outputs, targets)} | {loss_func.gps_loss(outputs, targets)}, score - {loss_func.gps_score(outputs, targets)}")
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = loss_func.gps_score(predict, ground)
    return loss, meas

In [23]:
def test(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환
    
    output_list = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, _, index in tqdm(dataloader):
            data = data.to(device)
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            
            df_temp = pd.DataFrame()
            df_temp['phone'] = index[0]
            df_temp['millisSinceGpsEpoch'] = index[1]
            df_temp[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']] = outputs.to('cpu').numpy()
            output_list.append(df_temp)
    
    predicts = pd.concat(output_list)
    return predicts
            

In [ ]:
train_loss_list = []
train_meas_list = []
valid_loss_list = []
valid_meas_list = []

patience_count = 0
min_valid_meas = np.inf
checkpoint_name = ""

epoch_prog = tqdm(range(hyper_parameters['nepochs']), position = 0, desc = "EPOCH")
epoch_status_log = tqdm(total=0, position = 2, bar_format='{desc}')
train_status_log = tqdm(total=0, position = 3, bar_format='{desc}')
valid_status_log = tqdm(total=0, position = 4, bar_format='{desc}')

if not os.path.isdir(f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/")
    
for epoch in epoch_prog:
    train_prog = tqdm(train_loader, position = 5, desc = 'Train', leave = False)
    valid_prog = tqdm(valid_loader, position = 6, desc = 'Valid', leave = False)

    train_loss, train_meas = train(epoch, train_prog, train_status_log)
    valid_loss, valid_meas = valid(valid_prog, valid_status_log)
    
    if valid_meas < min_valid_meas:
        min_valid_meas = valid_meas
        checkpoint_name = f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/model-{epoch}-{min_valid_meas}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > hyper_parameters['max_patience_count']):
            break
    
    train_loss_list.append(train_loss)
    train_meas_list.append(train_meas)
    valid_loss_list.append(valid_loss)
    valid_meas_list.append(valid_meas)
    
    status = f"EPOCH:{epoch}/{hyper_parameters['nepochs']} train status: loss - {train_loss}, score - {train_meas}, valid status: loss - {valid_loss}, score - {valid_meas}"
    epoch_status_log.set_description_str(status)

    print(status)

history = dict()
history['train_loss'] = train_loss_list
history['train_meas'] = train_meas_list
history['valid_loss'] = valid_loss_list
history['valid_meas'] = valid_meas_list


EPOCH:   0%|          | 0/10000 [00:00<?, ?it/s]

Train:   0%|          | 0/468 [00:00<?, ?it/s]

Valid:   0%|          | 0/51 [00:00<?, ?it/s]

EPOCH:0/10000 train status: loss - 34.646759033203125, score - 2784431.75, valid status: loss - 21.035343170166016, score - 48211.5234375


Train:   0%|          | 0/468 [00:00<?, ?it/s]

...

Train:   0%|          | 0/468 [00:00<?, ?it/s]

Valid:   0%|          | 0/51 [00:00<?, ?it/s]

EPOCH:48/10000 train status: loss - 12.356148719787598, score - 18009.07421875, valid status: loss - 8.681280136108398, score - 15421.8203125


Train:   0%|          | 0/468 [00:00<?, ?it/s]

Valid:   0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(2,1,1)
plt.plot(history['train_loss'], label = 'train')
plt.plot(history['valid_loss'], label = 'valid')
plt.ylabel('loss')
plt.yscale('log')

plt.subplot(2,1,2)
plt.plot(history['train_meas'], label = 'train')
plt.plot(history['valid_meas'], label = 'valid')
plt.ylabel('meas')
plt.yscale('log')

#  Output

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)
submission.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_name))
torch.save(model.state_dict(), f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint/model-{epoch}-{min_valid_meas}.pth")

In [ ]:
predict = test(test_loader)
print(predict.shape)
predict.head()

In [ ]:
submission = submission[['phone', 'millisSinceGpsEpoch']].merge(predict[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']]
                                                                , on = ['phone', 'millisSinceGpsEpoch'])
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv(f"./models/{hyper_parameters['notebookName']}/result-{num_files}esult.csv", index = False)
pd.DataFrame([]).to_csv(PATH / f"{num_files}")